#### __LIBRARIES__

In [126]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

#### __LOAD DATASET__

In [127]:
df = pd.read_csv("WSdata.csv")
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=119/30464,..."
1,2,0.160909,192.167.7.43,192.167.7.45,UDP,162,8130 > 12345 Len=120
2,3,1.024022,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=120/30720,..."
3,4,1.161161,192.167.7.43,192.167.7.45,UDP,162,8131 > 12345 Len=120
4,5,2.048007,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=121/30976,..."


#### __EDA__

- __Dataset Overview__

In [128]:
print("Dataset Shape:", df.shape)

Dataset Shape: (19380, 7)


In [129]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19380 entries, 0 to 19379
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No.          19380 non-null  int64  
 1   Time         19380 non-null  float64
 2   Source       19380 non-null  object 
 3   Destination  19380 non-null  object 
 4   Protocol     19380 non-null  object 
 5   Length       19380 non-null  int64  
 6   Info         19379 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 1.0+ MB
None


- __Missing Values & Duplicates__

In [ ]:
df = df.drop(columns=["No."], errors="ignore")
df.head()

,Time,Source,Destination,Protocol,Length,Info
0,0.000000,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=119/30464,..."
1,0.160909,192.167.7.43,192.167.7.45,UDP,162,8130 > 12345 Len=120
2,1.024022,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=120/30720,..."
3,1.161161,192.167.7.43,192.167.7.45,UDP,162,8131 > 12345 Len=120
4,2.048007,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=121/30976,..."


#### __ENCODING & NORMALIZATION__

In [ ]:
protocol_encoder = LabelEncoder()
df['Protocol_Encoded'] = protocol_encoder.fit_transform(df['Protocol'])

scaler = MinMaxScaler()
df[['Time_Normalized', 'Length_Normalized']] = scaler.fit_transform(df[['Time', 'Length']])

df.head()

,Time,Source,Destination,Protocol,Length,Info,Protocol_Encoded,Time_Normalized,Length_Normalized
0,0.000000,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=119/30464,...",4,0.000000,0.019746
1,0.160909,192.167.7.43,192.167.7.45,UDP,162,8130 > 12345 Len=120,9,0.000555,0.042313
2,1.024022,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=120/30720,...",4,0.003534,0.019746
3,1.161161,192.167.7.43,192.167.7.45,UDP,162,8131 > 12345 Len=120,9,0.004007,0.042313
4,2.048007,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=121/30976,...",4,0.007067,0.019746


In [10]:
import joblib
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(protocol_encoder, 'protocol_encoder.pkl')

['protocol_encoder.pkl']

In [11]:
# Define: heuristic rules for labeling the traffic data
def label_traffic(row):
    """
    Assign '1' (malicious)
    Assign '0' (normal)
    """
    malicious_protocols = {'ICMP', 'SSH', 'MDNS', 'Telenet', 'TLSv1.2'} 
    malicious_keywords = {'cipher', 'syn', 'fin', 'psh', }

    # Check: protocol is suspicious
    if row['Protocol'] in malicious_protocols:
        return 1

    # Check: malicious keywords
    if any(keyword in str(row['Info']).lower() for keyword in malicious_keywords):
        return 1

    # otherwise, normal
    return 0

# Apply labeling function to dataset
df['Label'] = df.apply(label_traffic, axis=1)
df.head()

,Time,Source,Destination,Protocol,Length,Info,Protocol_Encoded,Time_Normalized,Length_Normalized,Label
0,0.000000,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=119/30464,...",4,0.000000,0.019746,1
1,0.160909,192.167.7.43,192.167.7.45,UDP,162,8130 > 12345 Len=120,9,0.000555,0.042313,0
2,1.024022,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=120/30720,...",4,0.003534,0.019746,1
3,1.161161,192.167.7.43,192.167.7.45,UDP,162,8131 > 12345 Len=120,9,0.004007,0.042313,0
4,2.048007,192.167.7.43,192.167.7.45,ICMP,98,"Echo (ping) request id=0x35a7, seq=121/30976,...",4,0.007067,0.019746,1


In [12]:
print(df['Label'].value_counts())

Label
0    16575
1     2805
Name: count, dtype: int64


In [13]:
# Select features and labels
features = ['Protocol_Encoded', 'Time_Normalized', 'Length_Normalized']
X = df[features]
y = df['Label']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
models = {
    "logistic regression": LogisticRegression(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "TabNet": TabNetClassifier(),
    "LightGBM": LGBMClassifier()
}

c:\Users\DeLL\Desktop\Network-Intrusion-Detection-Using-Wireshark\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [ ]:
# Train and evaluate each model
for name, model in models.items():
    # Convert DataFrames to NumPy arrays for models that require it
    model.fit(X_train.values, y_train.values)
    
    y_pred = model.predict(X_test.values)
    
    # Print model performance with better readability
    print("="*40)  # Adds a separator line
    print(f"{name} Model Performance:\n")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-score:", f1_score(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_pred))
    print(" "*40)

logistic regression Model Performance:

Accuracy: 0.8412452700378397
Precision: 0.0
Recall: 0.0
F1-score: 0.0
ROC AUC: 0.4908671216378964
                                        
CatBoost Model Performance:

Accuracy: 0.9950120399036808
Precision: 1.0
Recall: 0.9651442307692307
F1-score: 0.982262996941896
ROC AUC: 0.9825721153846154
                                        


c:\Users\DeLL\Desktop\Network-Intrusion-Detection-Using-Wireshark\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.291   |  0:00:03s
epoch 1  | loss: 0.07942 |  0:00:05s
epoch 2  | loss: 0.05239 |  0:00:07s
epoch 3  | loss: 0.04534 |  0:00:09s
epoch 4  | loss: 0.04137 |  0:00:10s
epoch 5  | loss: 0.04106 |  0:00:12s
epoch 6  | loss: 0.03582 |  0:00:13s
epoch 7  | loss: 0.0357  |  0:00:15s
epoch 8  | loss: 0.03728 |  0:00:16s
epoch 9  | loss: 0.03421 |  0:00:18s
epoch 10 | loss: 0.03585 |  0:00:19s
epoch 11 | loss: 0.03458 |  0:00:21s
epoch 12 | loss: 0.03594 |  0:00:22s
epoch 13 | loss: 0.0326  |  0:00:24s
epoch 14 | loss: 0.03174 |  0:00:25s
epoch 15 | loss: 0.03362 |  0:00:27s
epoch 16 | loss: 0.03769 |  0:00:28s
epoch 17 | loss: 0.03459 |  0:00:30s
epoch 18 | loss: 0.03481 |  0:00:31s
epoch 19 | loss: 0.03914 |  0:00:33s
epoch 20 | loss: 0.03695 |  0:00:34s
epoch 21 | loss: 0.03449 |  0:00:36s
epoch 22 | loss: 0.03136 |  0:00:37s
epoch 23 | loss: 0.03176 |  0:00:39s
epoch 24 | loss: 0.03228 |  0:00:41s
epoch 25 | loss: 0.02991 |  0:00:42s
epoch 26 | loss: 0.03088 |  0:00:44s
e

c:\Users\DeLL\Desktop\Network-Intrusion-Detection-Using-Wireshark\venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [17]:
import joblib

# Save CatBoost
joblib.dump(models["CatBoost"], "catboost_model.pkl")

# Save LightGBM
joblib.dump(models["LightGBM"], "lightgbm_model.pkl")

['lightgbm_model.pkl']